In [1]:
import pandas as pd

In [2]:
territoires_url = 'https://www.resultats-elections.interieur.gouv.fr/telechargements/LG2024/territoires/territoires.xml'

In [3]:
mapping_dept = (
    pd
    .read_csv('lg2022_circonscriptions.csv')
    .groupby(['CodDpt3Car', 'CodDpt'])
    .agg({'CodCirLg': len})
    .reset_index()
    .set_index('CodDpt3Car')
    [['CodDpt']]
    .query('CodDpt.str.startswith("Z")')
    .to_dict()
    ['CodDpt']
)

mapping_dept

{'099': 'ZZ',
 '971': 'ZA',
 '972': 'ZB',
 '973': 'ZC',
 '974': 'ZD',
 '975': 'ZS',
 '976': 'ZM',
 '977': 'ZX',
 '986': 'ZW',
 '987': 'ZP',
 '988': 'ZN'}

In [4]:
candidats = (
    pd
    .read_csv('https://static.data.gouv.fr/resources/elections-legislatives-des-30-juin-et-7-juillet-2024-candidatures/20240619-161410/legislatives-2024-candidatures-france-entiere-tour-1-2024-06-19-17h10.csv')
    .assign(
        NumCirc = lambda df: df['Code circonscription'].str[-2:].str.pad(3, fillchar="0"),
        Dept = lambda df: df['Code circonscription'].str[:-2].str.pad(2, fillchar="0").str.pad(2, fillchar="0"),
        CodCirc = lambda df: df.Dept.replace(mapping_dept) + df.NumCirc
    )
)

candidats

,Code département,Département,Code circonscription,Libellé circonscription,Numéro de panneau,N° dépôt,Sexe du candidat,Nom du candidat,Prénom du candidat,Date de naissance du candidat,...,Profession,Sortant,Sexe remplaçant,Nom remplaçant,Prénom remplaçant,Date de naissance remplaçant,Sortant remplaçant,NumCirc,Dept,CodCirc
0,1,Ain,101,1ère circonscription,1,2,M,LAHY,Éric,24/02/1966,...,"(34) - Professeur, profession scientifique",NaN,M,DUPONT,Jean Claude,06/07/1960,NaN,001,01,01001
1,1,Ain,101,1ère circonscription,2,4,M,MAÎTRE,Christophe,22/04/1969,...,(22) - Commerçant et assimilé,NaN,M,MOUREY,Jean-Marie,03/01/1965,NaN,001,01,01001
2,1,Ain,101,1ère circonscription,3,3,M,BRETON,Xavier,25/11/1962,...,(33) - Cadre de la fonction publique,OUI,M,FLOCHON,Jean-Yves,15/08/1958,NaN,001,01,01001
3,1,Ain,101,1ère circonscription,4,5,M,GUERAUD,Sébastien,09/09/1973,...,"(34) - Professeur, profession scientifique",NaN,F,DIMBERTON,Gaëlle,12/01/1984,NaN,001,01,01001
4,1,Ain,101,1ère circonscription,5,1,M,VINCENT,Cyril,21/10/1982,...,(23) - Chef d'entreprise de 10 salariés ou plus,NaN,M,VINCENT,Christian,30/08/1955,NaN,001,01,01001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4005,ZZ,Français établis hors de France,ZZ11,11ème circonscription,11,7,M,CHEMINADE,Jacques,20/08/1941,...,(74) - Ancien cadre,NaN,F,BIERRE,Christine,19/04/1950,NaN,011,ZZ,ZZ011
4006,ZZ,Français établis hors de France,ZZ11,11ème circonscription,12,8,M,ASSELINEAU,François,14/09/1957,...,(33) - Cadre de la fonction publique,NaN,F,NGUYEN,Fleur,09/06/1971,NaN,011,ZZ,ZZ011
4007,ZZ,Français établis hors de France,ZZ11,11ème circonscription,13,1,F,GENETET,Anne,20/04/1963,...,(31) - Profession libérale,OUI,M,PROVENDIER,Remi,24/12/1971,NaN,011,ZZ,ZZ011
4008,ZZ,Français établis hors de France,ZZ11,11ème circonscription,14,15,M,CRUCIANI,Jordan,24/09/1990,...,(33) - Cadre de la fonction publique,NaN,M,DAPRILE,Rafaele,06/07/2005,NaN,011,ZZ,ZZ011


In [5]:
candidats.to_csv('lg2024_t1_candidats.csv', index=False)

In [6]:
nuances2024 = (
    candidats
    .pivot_table(
        index = 'CodCirc',
        columns = 'Code nuance',
        values = 'Nom du candidat',
        aggfunc= lambda x : len(x) >= 1
    )
    .fillna(False)
)

nuances2024

Code nuance,COM,DIV,DSV,DVC,DVD,DVG,ECO,ENS,EXD,EXG,...,LR,RDG,REC,REG,RN,SOC,UDI,UG,UXD,VEC
CodCirc,,,,,,,,,,,,,,,,,,,,,
01001,False,False,True,False,False,False,False,True,False,True,...,True,False,False,False,True,False,False,True,False,False
01002,False,False,False,True,False,False,False,False,True,True,...,True,False,False,False,True,False,False,True,False,False
01003,False,True,True,False,False,False,False,True,False,True,...,True,False,False,False,True,False,False,True,False,False
01004,False,True,False,False,False,False,False,True,False,True,...,True,False,False,False,True,False,False,True,False,False
01005,False,False,False,False,True,False,True,True,False,True,...,True,False,True,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZ007,False,True,False,True,False,True,False,True,False,False,...,True,False,True,False,True,False,False,True,False,False
ZZ008,False,True,False,True,True,True,False,True,False,False,...,True,False,True,False,False,False,False,True,False,False
ZZ009,False,True,True,True,True,True,False,True,False,False,...,True,False,True,False,True,False,False,True,False,False


In [7]:
nuances2024.to_csv('lg2024_t1_nuances.csv')